<a href="https://colab.research.google.com/github/centeno/covid-impact-scrapper/blob/master/step01_collection_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SEMPRE RODAR ANTES

### Instalação das dependencias

In [1]:
#!apt-get update # to update ubuntu to correctly run apt install

# scraper
#!pip install selenium
#!apt install chromium-chromedriver
#!pip install webdriver-manager
#!cp /usr/lib/chromium-browser/chromedriver/usr/bin

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium msticpy tld folium
!pip pymongo pymongo[srv] dnspython

# geocoder ipaddress
#!pip install msticpy

# get domain
#!pip3 install tld


# spacy
!pip install -U pip setuptools wheel
!pip install -U spacy
#!python -m spacy download en
#!python -m spacy download pt
!python -m spacy download en_core_web_sm
# spacy contextual Spell Check


!pip install contextualSpellCheck
!curl https://raw.githubusercontent.com/codelucas/newspaper/master/download_corpora.py | python3


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubun

### Import das dependencias

In [2]:
#from kora.selenium import wd as driver

import os
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, date, timedelta

import urllib
import socket
from msticpy.sectools.geoip import IPStackLookup

import spacy
import contextualSpellCheck

import pymongo
import pandas as pd
from datetime import datetime as dt
from datetime import date, timedelta
from pandas.tseries.offsets import DateOffset
import dns

import tld
from tld import get_tld, get_fld

from google.colab import drive

### Connect google drive

In [3]:
# Codigo para montar o Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/My Drive/covid19impact/data/2021/01_collect/en

/content/drive/My Drive/covid19impact/data/2021/01_collect/en


In [5]:
# Listar os arquivos da pasta do projeto no Google Drive
os.chdir("/content/drive/My Drive/covid19impact/data/2021/01_collect/en/")
#!ls

### Import chaves de api

In [6]:
#home
keys = pd.read_csv("/content/drive/My Drive/covid19impact/keys/credentials.csv", sep=';')
access_token_ipstack = keys.loc[0,"key"]
access_token_pymongo = keys.loc[1,"key"]

### Criação das funções

In [7]:
import pandas as pd
GSE_RESULTS_CLASS = 'ZINbbc xpd O9g5cc uUPGi' # 'g' # 'tF2Cxc' # rc
GSE_MAX_RESULTS = 200

NLP = spacy.load("en_core_web_sm")
contextualSpellCheck.add_to_pipe(NLP)

def get_driver():
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36')    
  return webdriver.Chrome('chromedriver',options=options)

def page_source(query, date):
  d = date.strftime('%Y%m%d')
  q = "{} after:{} and before:{}".format(query, date.strftime('%Y-%m-%d'), date.strftime('%Y-%m-%d'))
  url = "https://www.google.com/search?q={}&sort=date:r:{}:{}&num={}".format(q, d, d, GSE_MAX_RESULTS)
  driver = get_driver()
  driver.get(url)
  soup = BeautifulSoup(driver.page_source, 'html.parser') 
  return driver.page_source
  
def fetch_result(date, item):
  try:
    return {
        'date': date.strftime('%Y-%m-%d'),
        'header' : item.select_one('h3').text,
        'summary': item.findChildren('div', recursive=False)[-1].text,
        'link' : item.find('a', href=True)['href'].replace('/url?q=', ''),
    }
  except:
    return None

def search(query, date):
  soup = BeautifulSoup(page_source(query, date), 'html.parser') 
  html_results = soup.find_all('div', {'class': GSE_RESULTS_CLASS })
  results = [fetch_result(date, r) for r in html_results]
  return pd.DataFrame(list(filter(None, results)))

def fetch_domain(df):
  df['protocol'],df['domain'],df['path'],df['query'],df['fragment'] = zip(*[urllib.parse.urlsplit(x) for x in df.link])
  return df

def fetch_hostname(df): 
  urls = df['link']
  hostnames = []
  for url in urls:
    try:
      res = get_tld(url, as_object=True)
      hostnames.append(res.fld)
    except:
      res = np.nan
      hostnames.append(res)
  df['host_name'] = hostnames
  return df

def fetch_ipaddress(df):
  domains = df['domain']
  ipaddress = []
  for i in domains:
      i_strip = i.strip()
      try:
          ip = socket.gethostbyname(i_strip)
          ipaddress.append(ip)
      except socket.gaierror:
        ipaddress.append('127.0.0.1')
  df['ip_address'] = ipaddress
  return df

def fetch_location(df):
  iplocation = IPStackLookup(api_key=access_token_ipstack)
  df = iplocation.df_lookup_ip(df, column="ip_address")
  return df

def fetch_nlp(df):
  df['header_context_true'] = df['header'].apply(lambda x: str(NLP(x)))
  df['summary_context_true'] = df['summary'].apply(lambda x: str(NLP(x)))
  return df

def save(df):
  client = pymongo.MongoClient(access_token_pymongo)
  db = client.covid_impact
  df = df.astype('str')
  db.impacts.insert_many(df.to_dict('records'))
  

def load():
  client = pymongo.MongoClient(access_token_pymongo)
  db = client.covid_impact
  df = pd.DataFrame.from_records(db.impacts.find())
  return df

### Merge csv's or Create a new scraper

In [8]:
df = load()
df.tail()

,_id,date,header,summary,link,header_context_true,summary_context_true,protocol,domain,path,query,fragment,host_name,ip_address,level_0,index,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,edges,Type,AdditionalData,IpAddress
1607,6070bcbacde398f9477906e5,2021-03-16,"Esri Disaster Response Program | Maps, Softwar...","When a disaster or crisis strikes, we're here ...",https://www.esri.com/en-us/disaster-response/o...,"Esri Disaster Response Program | Maps, Softwar...","When a disaster or crisis strikes, we're here ...",https,www.esri.com,/en-us/disaster-response/overview&sa=U&ved=2ah...,,,esri.com,184.25.191.207,101,101,US,United States,Virginia,Ashburn,-77.47419738769531,39.043701171875,16625.0,set(),geolocation,{},184.25.191.207
1608,6070bcbacde398f9477906e6,2021-03-16,The Impact of COVID-19 and Racism on Faith at ...,"Additionally, the COVID-19 pandemic has shed a...",https://rplp.rice.edu/research-studies/impact-...,The Impact of COVID-19 and Racism on Faith at ...,"Additionally, the COVID-19 pandemic has shed a...",https,rplp.rice.edu,/research-studies/impact-covid-19-and-racism-f...,,,rice.edu,52.1.253.98,102,102,US,United States,Virginia,Ashburn,-77.47419738769531,39.043701171875,14618.0,set(),geolocation,{},52.1.253.98
1609,6070bcbacde398f9477906e7,2021-03-16,COVID-19: the impact on undeclared work and ke...,"Jun 9, 2020 · The European Platform tackling u...",https://ec.europa.eu/social/main.jsp%3FlangId%...,COVID-19: the impact on undeclared work and ke...,"Jun 9, 2020 · The European Platform tackling u...",https,ec.europa.eu,/social/main.jsp%3FlangId%3Den%26catId%3D89%26...,,,europa.eu,147.67.210.30,103,103,LU,Luxembourg,Luxembourg,Luxembourg,6.1062397956848145,49.616241455078125,42848.0,set(),geolocation,{},147.67.210.30
1610,6070bcbacde398f9477906e8,2021-03-16,COVID-19 Update: Approach and Systems Response...,At Co-Impact – as in every organization and co...,https://www.co-impact.org/covid-19-update-appr...,COVID-19 Update: Approach and Systems Response...,At Co-Impact – as in every organization and co...,https,www.co-impact.org,/covid-19-update-approach-and-systems-response...,,,co-impact.org,162.241.224.68,104,104,US,United States,Utah,Provo,-111.6124267578125,40.21580123901367,46606.0,set(),geolocation,{},162.241.224.68
1611,6070bcbacde398f9477906e9,2021-03-16,COVID-19's impact on tech companies: The good ...,"Amid the layoffs, furloughs, and pay cuts at s...",https://www.ringcentral.com/us/en/blog/covid-i...,COVID-19's impact on tech companies: The good ...,"Amid the layoffs, furloughs, and pay cuts at s...",https,www.ringcentral.com,/us/en/blog/covid-impact-on-tech-startups/&sa=...,,,ringcentral.com,69.192.178.8,105,105,US,United States,Virginia,Ashburn,-77.47419738769531,39.043701171875,16625.0,set(),geolocation,{},69.192.178.8


In [9]:
len_files = df.shape[0]
len_files

1612

In [10]:
# get last date da tabela
len_files = len(df)

if len_files >= 1:
  df['date'] = pd.to_datetime(df['date'], dayfirst = False)
  last_date_table = df['date'].max()
  next_date = (last_date_table.date() + DateOffset(days = 1)).date()
  print('Last date is: ' + str(last_date_table.date()))
  print('Next date is: ' + str(next_date))
  print('\n')
  print('Shape: ' + str(df.shape))
else:
    # get first date
    next_date = date(2019, 9, 30)
    print(next_date)

Last date is: 2021-03-16
Next date is: 2021-03-17


Shape: (1612, 27)


## SCRAPER

In [11]:
keyword = '"covid-19" OR "covid" OR "coronavirus" AND “impact" OR "disaster" OR "resilience" AND “software” OR "software engineer" OR "agile" OR "information industry" OR "IT" OR "disaster informatics" OR "crisis informatics" OR "productivity" OR "remote" OR "work" OR "future of work"'

if len_files >= 1:
  date = next_date 
else:
  date = datetime(2019, 9, 30)

In [12]:
#get seach
df_new = search(keyword, date)

#get domain
df_new = fetch_domain(df_new)

#get hostname
df_new = fetch_hostname(df_new)

#get ipaddress
df_new = fetch_ipaddress(df_new)

#get geocoded ipaddress
df_new = fetch_location(df_new)

In [13]:
#get nlp contextualSpellCheck 
df_new = fetch_nlp(df_new)

In [14]:
#save df new
save(df_new)

In [15]:
#print
df_new.head()

,date,header,summary,link,protocol,domain,path,query,fragment,host_name,ip_address,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,edges,Type,AdditionalData,IpAddress,header_context_true,summary_context_true
0,2021-03-17,"COVID-19's impact on work, workers, and the wo...","Sep 25, 2020 · What will the world of work loo...",https://business.cornell.edu/hub/2020/09/25/co...,https,business.cornell.edu,/hub/2020/09/25/covid-19s-impact-work-workers-...,,,cornell.edu,23.185.0.3,US,United States,California,San Francisco,-122.408730,37.790939,54113.0,{},geolocation,{},23.185.0.3,"COVID-19's impact on work, workers, and the wo...","Sep 25, 2020 · What will the world of work loo..."
1,2021-03-17,"The impact of Covid-19 on productivity | VOX, ...","Jan 18, 2021 · It estimates total factor produ...",https://voxeu.org/article/impact-covid-19-prod...,https,voxeu.org,/article/impact-covid-19-productivity&sa=U&ved...,,,voxeu.org,104.26.13.78,US,United States,California,San Jose,-121.838226,37.330528,13335.0,{},geolocation,{},104.26.13.78,"The impact of Covid-19 on productivity | VOX, ...","Jan 18, 2021 · It estimates total factor produ..."
2,2021-03-17,Agile resilience: Lessons from COVID-19 | McKi...,"Oct 13, 2020 · However, understandably, no org...",https://www.mckinsey.com/business-functions/or...,https,www.mckinsey.com,/business-functions/organization/our-insights/...,,,mckinsey.com,23.207.208.133,US,United States,Virginia,Linton Hall,-77.489113,38.746220,16625.0,{},geolocation,{},23.207.208.133,Agile resilience: Lessons from COVID-19 | McKi...,"Oct 13, 2020 · However, understandably, no org..."
3,2021-03-17,Agile resilience: Lessons from COVID-19 | McKi...,"Oct 13, 2020 · However, understandably, no org...",https://www.mckinsey.com/business-functions/or...,https,www.mckinsey.com,/business-functions/organization/our-insights/...,,,mckinsey.com,23.207.208.133,US,United States,Virginia,Linton Hall,-77.489113,38.746220,16625.0,{},geolocation,{},23.207.208.133,Agile resilience: Lessons from COVID-19 | McKi...,"Oct 13, 2020 · However, understandably, no org..."
4,2021-03-17,Coronavirus' business impact: Evolving perspec...,"3 days ago · In the new world of work, which a...",https://www.mckinsey.com/business-functions/ri...,https,www.mckinsey.com,/business-functions/risk/our-insights/covid-19...,,,mckinsey.com,23.207.208.133,US,United States,Virginia,Linton Hall,-77.489113,38.746220,16625.0,{},geolocation,{},23.207.208.133,Coronavirus' business impact: Evolving perspec...,"3 days ago · In the new world of work, which a..."


In [16]:
df = load()
df.tail()

,_id,date,header,summary,link,header_context_true,summary_context_true,protocol,domain,path,query,fragment,host_name,ip_address,level_0,index,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,edges,Type,AdditionalData,IpAddress
1713,6070bda176f4eb45688daf80,2021-03-17,COVID-19: Impact on Education Equity - Resourc...,"So while school leaders, teachers, and parents...",https://edtrust.org/covid-19-impact-on-educati...,COVID-19: Impact on Education Equity - Resourc...,"So while school leaders, teachers, and parents...",https,edtrust.org,/covid-19-impact-on-education-equity-resources...,,,edtrust.org,104.20.66.117,NaN,NaN,US,United States,California,San Jose,-121.83822631835938,37.330528259277344,13335.0,set(),geolocation,{},104.20.66.117
1714,6070bda176f4eb45688daf81,2021-03-17,Responding to COVID-19's Impact on Resilience ...,"In a vicious cycle, shocks to nutrition from m...",https://www.usaid.gov/who-we-are/organization/...,Responding to COVID-19's Impact on Resilience ...,"In a vicious cycle, shocks to nutrition from m...",https,www.usaid.gov,/who-we-are/organization/bureaus/bureau-resili...,,,usaid.gov,104.82.11.192,NaN,NaN,US,United States,Virginia,Linton Hall,-77.4891128540039,38.746219635009766,16625.0,set(),geolocation,{},104.82.11.192
1715,6070bda176f4eb45688daf82,2021-03-17,"Esri Disaster Response Program | Maps, Softwar...","When a disaster or crisis strikes, we're here ...",https://www.esri.com/en-us/disaster-response/o...,"Esri Disaster Response Program | Maps, Softwar...","When a disaster or crisis strikes, we're here ...",https,www.esri.com,/en-us/disaster-response/overview&sa=U&ved=2ah...,,,esri.com,184.25.191.207,NaN,NaN,US,United States,Virginia,Ashburn,-77.47419738769531,39.043701171875,16625.0,set(),geolocation,{},184.25.191.207
1716,6070bda176f4eb45688daf83,2021-03-17,The Impact of COVID-19 and Racism on Faith at ...,"Additionally, the COVID-19 pandemic has shed a...",https://rplp.rice.edu/research-studies/impact-...,The Impact of COVID-19 and Racism on Faith at ...,"Additionally, the COVID-19 pandemic has shed a...",https,rplp.rice.edu,/research-studies/impact-covid-19-and-racism-f...,,,rice.edu,52.1.253.98,NaN,NaN,US,United States,Virginia,Ashburn,-77.47419738769531,39.043701171875,14618.0,set(),geolocation,{},52.1.253.98
1717,6070bda176f4eb45688daf84,2021-03-17,COVID-19: the impact on undeclared work and ke...,"Jun 9, 2020 · The European Platform tackling u...",https://ec.europa.eu/social/main.jsp%3FlangId%...,COVID-19: the impact on undeclared work and ke...,"Jun 9, 2020 · The European Platform tackling u...",https,ec.europa.eu,/social/main.jsp%3FlangId%3Den%26catId%3D89%26...,,,europa.eu,147.67.34.30,NaN,NaN,LU,Luxembourg,Luxembourg,Luxembourg,6.1062397956848145,49.616241455078125,42848.0,set(),geolocation,{},147.67.34.30
